In [ ]:
from arcgis.gis import GIS
import pandas as pd

In [ ]:
gis = GIS('<ORGANIZATION>', '<USERNAME>','<PASSWORD>')

In [ ]:
#for deleting multiple users from a list in a csv file
my_file = '<FILE PATH>'
my_csv = pd.read_csv(my_file)
username = my_csv['username'] #assumes that the column of usernames is called 'username'. adjust as necessary

### TRANSFER GROUP OWNERSHIP

In [ ]:
validation_variable = False

for u in username:
    #print(u)
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)): 
        if gisusers[p].username == u: 
            validation_variable = True 
            user = gisusers[p]
            groups = user.groups 

            for x in range(len(groups)):
                try: 
                    owner = groups[x].owner 
                    if owner == u:
                        groups[x].reassign_to('<NEW ACCOUNT>') 

                except:
                    print('ERROR with ' + user.username)
                    continue
                    
    if validation_variable == False:
        print("Search did not successfully locate " + u + ". Please manually remove from groups.")

### TRANSFER ITEMS

In [ ]:
#transfers items one by one to keep folder structure of the user

validation_variable = False

for u in username:
    #print('current user: ' + u)
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)):
        if gisusers[p].username == u: #check that search returned the right person
            validation_variable = True #if this doesn't trigger, an error message will tell us that user wasn't found
            user = gisusers[p]

            folders_info = user.folders
            folders = []
            all_items = user.items() #all items (in and out of folders) that a user owns

            for x in range(len(folders_info)):
                folders.append(folders_info[x]['title']) #creates list of folder names for user

            for i in range(len(folders)):
                items = user.items(folder = folders[i])
                #print('current folder: ' + folders[i])
                for j in range(len(items)):
                    #print(items[j])
                    try:
                        items[j].update(item_properties={'title': items[j].title + '_' + items[j].type})
                        folder_name = u + '_' + folders[i] #new folder name = old folder name + user
                        items[j].reassign_to(target_owner = 'GIS_Help_audubon', target_folder=folder_name) #reassigns item
                    except:
                        #when errors appear, check that user's items on AGOL to see if it needs to be manually moved
                        #Most errors seem to be a glitch, but if item is in a shared update group
                        #it will need to be moved by hand
                        print('ERROR for user ' + user.username + ' in a folder.')
                        print('    folder: ' + folders[i])
                        print('    item: ' + items[j].title)
                        continue

            for k in range(len(all_items)):
                #print(all_items[k])
                try:
                    all_items[k].update(item_properties={'title': all_items[k].title + '_' + all_items[k].type})
                    folder_name = u + "_home" #new folder name
                    all_items[k].reassign_to(target_owner = 'GIS_Help_audubon', target_folder=folder_name) #reassigns items
                except:
                    #Be sure to check errors as they appear (on AGOL)
                    #Most are nothing, but if item is in a shared update group, needs to be moved by hand
                    print('ERROR for user ' + user.username + ' in the home directory.')
                    print('    item: ' + all_items[k].title)
                    continue
    if validation_variable == False:
        print("Search did not successfully locate " + u + ". Please manually transfer items.")

### REMOVING LICENSES

In [ ]:
#find everyone with pro/extension licenses
pro_license = gis.admin.license.get('ArcGIS Pro')
all_license = pro_license.all()

validation_variable = False
#makes a list of all users in org with licenses
licenseUsers = []
for x in range(len(all_license)):
    licenseUsers.append(all_license[x]['username'])
    
HasLicenseInList = set(username).intersection(licenseUsers) #finds the users that are in both the license list and the deletion list

for u in HasLicenseInList:
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)):
        if gisusers[p].username == u:
            validation_variable = True
            user = gisusers[p]
            pro_license.revoke(username=user.username, entitlements='*') #removes all licenses
            #print('license has been removed for ' + user.username)

    if validation_variable == False:
        print("Search did not successfully locate " + u + ". Please manually transfer items.")

In [ ]:
pro_license = gis.admin.license.get('ArcGIS Pro')
all_license = pro_license.all()

licenseUsers = []
for x in range(len(all_license)):
    licenseUsers.append(all_license[x]['username'])

In [ ]:
all_license

In [ ]:
all_advanced = []
for x in range(len(all_license)):
    all_advanced.append(all_license[x])

In [ ]:
all_advanced

In [ ]:
for x in range(len(all_advanced)):
    entitlements = all_advanced[x]['entitlements']
    entitlements = [sub.replace('desktopBasicN','desktopAdvN') for sub in entitlements]
    entitlements = [sub.replace('desktopStdN','desktopAdvN') for sub in entitlements]

    all_advanced[x]['entitlements'] = entitlements

In [ ]:
all_license

In [ ]:
for x in range(len(licenseUsers)):
    pro_license.revoke(username=licenseUsers[x], entitlements='*')

In [ ]:
for x in range(len(licenseUsers)):
    pro_license.assign(username=licenseUsers[x],entitlements=all_advanced[x]['entitlements'])

### DELETING USERS

In [ ]:
validation_variable = False
for u in username:
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)):
        if gisusers[p].username == u:
            try:
                validation_variable = True
                user = gisusers[p]
                #print(user.username)
                user.delete() #deletes user
                print(user.username + " has been deleted.")
            except:
                print('ERROR deleting ' + user.username)
                continue
    if validation_variable == False:
        print("Search did not successfully locate " + u + ". Please manually delete user.")

### Check that all users are gone

In [ ]:
#runs through deletion list
#prints username if still found in org, otherwise continues
for u in username:
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)):
        try:
            if gisusers[p].username == u:
                print(u + ' still exists!')
        except:
            continue

### Change Creators to Viewers

In [ ]:
for u in username:
    gisusers = gis.users.search(u)
    for p in range(len(gisusers)):
        try:
            if gisusers[p].username == u:
                gisusers[p].update_role(role = 'iAAAAAAAAAAAAAAA') #the code for a creator with a viewer role, must be updated before changing user type
                gisusers[p].update_license_type(user_type='viewer')
            if gisusers[p].level != '1':
                print("error with " + u + ". Please check their account.")
        except:
            print("error with " + u + ". Change by hand.")
            continue

In [ ]:
gisuser = gis.users.search('estrell_audubon')
gisuser[0].update_role(role = 'iAAAAAAAAAAAAAAA')

In [ ]:
gisusers[0].update_license_type(user_type='viewer')

In [ ]:
username

In [ ]:
test_dict = {}

In [ ]:
test_dict.update({'b':'2'})

In [ ]:
test_dict